# Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
RAG: models which combine pre-trained parametric and non-parametric memory for language generation

- **Parametric knowledge**: Learned during training that is implicitly stored in the neural network's weights.
- **Non-parametric knowledge**: Stored in an external knowledge source, such as a vector database.

For language generation tasks, we find that RAG models generate more specific, diverse and factual language than a state-of-the-art parametric-only seq2seq baseline.



Requirements:
- langchain for orchestration
- openai for the embedding model and LLM
- weaviate-client for the vector database

In [1]:
import dotenv
dotenv.load_dotenv()

True

In [2]:
import requests
from langchain.document_loaders import TextLoader

url = "https://raw.githubusercontent.com/hwchase17/chroma-langchain/88bd99222d46f763957c2873e48bbba2f3e4b36a/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
    f.write(res.text)

loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

In [ ]:
import weaviate
client = weaviate.connect_to_local(skip_init_checks=True)


In [4]:
# !pip install -U langchain-community

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

In [26]:
import os
cohere_api_key = os.getenv("COHERE_API_KEY")

In [27]:
import weaviate.classes.config as wvc
client.collections.delete("RAGChunks")

client.collections.create(
    name="RAGChunks",
    description="Chunks with externally-generated embeddings",
    vectorizer_config=wvc.Configure.Vectorizer.none(),
    properties=[
        wvc.Property(name="text", data_type=wvc.DataType.TEXT),
    ],
)


c:\Users\naina\anaconda3\Lib\site-packages\weaviate\collections\classes\config.py:1975: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for cls_field in self.model_fields:


In [8]:
#!pip install -U langchain-huggingface


In [28]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [10]:
#%pip install --upgrade langchain-weaviate weaviate-client>=4.0.0

In [29]:
from langchain_weaviate import WeaviateVectorStore
from langchain_core.documents import Document

vectorstore = WeaviateVectorStore(
    client=client,
    index_name="RAGChunks",
    text_key="text",
    embedding=embedding,  
    attributes=["source"]  # Include metadata fields
)


In [35]:
vectorstore.add_documents(documents=chunks)
retriever = vectorstore.as_retriever()

In [36]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nUse three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:\n"), additional_kwargs={})]


In [37]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct",temperature=0)

In [38]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)


In [39]:
query = "What did they say about Ukrainians?"
rag_chain.invoke(query)

'The context describes Ukrainians as a "proud" people who have shown they will not tolerate anyone trying to take their country backwards, and mentions their fight for freedom. It also highlights their courage in fighting back against Russian aggression. The speaker expresses support for the Ukrainians, stating that the US is providing over $1 Billion in direct assistance.'